## GAN model

In [6]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

In [7]:
import keras
import data_functions
import numpy
#Load data from already created pickle file
words, X, dataX, dataY, n_words, n_vocab, index2word, word2index = data_functions.load_from_pickle("data.pickle")

In [8]:
print(index2word[1])
print(index2word[n_vocab-1])

من
يوسوس


In [9]:
#Uncomment and Run this is plot_model does not work
import os
os.environ["PATH"] =  os.environ["PATH"] + os.pathsep + 'C:\\Program Files (x86)\\Graphviz2.38\\bin'

def show_image(imagefile, w, h):
    %pylab inline
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    img=mpimg.imread(imagefile)
    plt.figure(figsize=(w, h))
    imgplot = plt.imshow(img);
    plt.show();

def show_model(model, w, h):
    from keras.utils.vis_utils import plot_model
    plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True);
    show_image('model_plot.png', w, h)

In [23]:
class GAN():
    def __init__(self):
#         self.img_rows = 28
#         self.img_cols = 28
#         self.channels = 1
#         self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        self.input_shape = 100
        self.noise_dim = 50
        self.img_shape = (self.input_shape,1)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.noise_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (self.noise_dim,)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

#         print(model.summary())
#         show_model(model, 20, 20)

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

#         img_shape = (self.img_rows, self.img_cols, self.channels)
        img_shape = self.img_shape

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))   #, input_shape=img_shape
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
#         print(model.summary())
#         show_model(model, 10, 10)

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, X_train, epochs, batch_size=128, save_interval=50):

        # Load the dataset
#         (X_train, _), (_, _) = mnist.load_data()

#         # Rescale -1 to 1
#         X_train = (X_train.astype(np.float32) - 127.5) / 127.5
#         X_train = np.expand_dims(X_train, axis=3)
#         print('X_train shape=',X_train.shape)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]
#             print('imgs shape=',imgs.shape)

            noise = np.random.normal(0, 1, (half_batch, self.noise_dim))
#             print('noise shape=',noise.shape)
            
            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.noise_dim))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r = 5
        noise = np.random.normal(0, 1, (r , self.noise_dim))
        gen_imgs = self.generator.predict(noise)
        pred = np.reshape(gen_imgs * float(n_vocab), (r,-1))
        pred[pred < 0] = 0
        pred[pred > (n_vocab-1)] = 0
        pred = pred.astype(int)        
        for p in pred:
            s = [index2word[i+1] for i in p]
            s = ' '.join(s)
            print(s)
            print('----------------------')

#         print(pred)

#         # Rescale images 0 - 1
#         gen_imgs = 0.5 * gen_imgs + 0.5

#         fig, axs = plt.subplots(r, c)
#         cnt = 0
#         for i in range(r):
#             for j in range(c):
#                 axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
#                 axs[i,j].axis('off')
#                 cnt += 1
#         fig.savefig("./images/mnist_%d.png" % epoch)
#         plt.close()


In [24]:
gan = GAN()

In [25]:
gan.train(X[0:200], epochs=30000, batch_size=32, save_interval=200)

TypeError: train_on_batch() got an unexpected keyword argument 'callbacks'